# 一連の流れのサンプル

- 各種設定
- ベストパラメータ探索
    - fold out法で学習
    - early stopping有
    - 3回の平均値を精度とする
 
- ベストパラメータで交差検証

In [1]:
import sys
sys.path.append("..")
from data.data_loader import cnn_data_loader_cv
from models.set_model import CNN_base, D1D2_base

In [ ]:
from torch_geometric.utils import from_networkx, to_networkx
from glob import glob
from tqdm import tqdm

import torch
import networkx as nx


def make_datasets(x="degree"):

    data_list = []
    for i, file in enumerate(sorted(glob("/workspace/my_data/*"))):
        # adjlist のパスを取得
        paths = glob(file + "/*.adjlist")
        data_list += [data_from_adjlist(p, i, x) for p in tqdm(paths)]
    return data_list


# adjlist -> data (torch geometric)
def data_from_adjlist(path, label, x="degree"):
    """adjlist to torch geometric data"""
    # load adjlist
    G = nx.read_adjlist(path)
    # graph to torch geometric data
    data = from_networkx(G)

    # set node attribute
    if x == "degree":
        data.x = torch.tensor(
            [[d] for d in list(dict(G.degree()).values())],
            dtype=torch.float,
        )
    else:
        data.x = torch.tensor([[float(1)] for i in range(nx.number_of_nodes(G))])

    # set graph label
    data.y = torch.tensor([label])
    return data


In [2]:
model = D1D2_base("simple", 4, 100, 10)

In [3]:
model

D1D2_model(
  (inception): D1D2Conv(
    (conv1): Sequential(
      (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(1, 10, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (1): ReLU()
    )
    (e2econv): Sequential(
      (0): E2EBlock(
        (cnn1): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
        (cnn2): Conv2d(1, 10, kernel_size=(100, 1), stride=(1, 1))
      )
      (1): ReLU()
    )
    (pool): MaxPool2d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
    (softmax): Softmax(dim=0)
  )
  (classifier): Simple_classifier(
    (classifier): Sequential(
      (0): Linear(in_features=4000, out_features=4, bias=True)
      (1): Softmax(dim=1)
    )
  )
)

In [5]:
import torch.nn as nn
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [4]:
model(torch.ones(1,1,100,100))

NameError: name 'torch' is not defined

In [6]:
import torch
model(torch.ones(1,1,100,100))

tensor([[0.2512, 0.2507, 0.2525, 0.2457]], grad_fn=<SoftmaxBackward>)

In [16]:
loss = criterion(out, torch.tensor([1]))
loss

tensor(0.7045, grad_fn=<NllLossBackward>)

In [21]:
import os
os.makedirs("sample", exist_ok=True)

In [ ]:
loss = criterion(outputs, labels)

In [4]:
from experiments.cross_validation import CV

In [10]:
config = {
    "name" : "MUTAG",
    "model_name" : "CNN",
    "input_size" : 100,
    "adam_lr" : 0.01,
    "batchsize" : 32,
    "num_epoch" : 10,
    "trial" : 1, # 1fold に対して何回学習を行うか = 初期重みによるばらつきを削減
    "save_dir" : "./"
}

In [9]:
CV(num_class, dataset, "./sample", config)

AttributeError: 'dict' object has no attribute 'model_name'